In [ ]:
from datetime import datetime
from collections import defaultdict
import calendar

def generate_monthly_bill(item_list, target_month):
    target_year, target_mon = map(int, target_month.split('-'))
    first_day = datetime(target_year, target_mon, 1)
    last_day = datetime(target_year, target_mon, calendar.monthrange(target_year, target_mon)[1])
    total_days_in_month = (last_day - first_day).days + 1
    def parse_date(date_str):
        return datetime.strptime(date_str, "%Y-%m-%d") if date_str else None

    line_items = defaultdict(lambda: {
        "item_code": None,
        "rate": None,
        "qty": 0,
        "amount": 0.0,
        "billing_period": None
    })
    for item in item_list:
        start_date = parse_date(item['start_date'])
        stop_date = parse_date(item['stop_date'])
        rate = float(item['rate'])
        qty = int(item['qty'])

        if not start_date or not stop_date:
            continue

        overlap_start = max(start_date, first_day)
        overlap_end = min(stop_date, last_day)

        if overlap_start <= overlap_end:
            active_days = (overlap_end - overlap_start).days + 1
            billing_period = f"{overlap_start.strftime('%Y-%m-%d')} to {overlap_end.strftime('%Y-%m-%d')}"
            item_amount = rate * qty * (active_days / total_days_in_month)
            group_key = (item['item_code'], rate, billing_period)
            line_items[group_key]["item_code"] = item["item_code"]
            line_items[group_key]["rate"] = rate
            line_items[group_key]["qty"] += qty
            line_items[group_key]["amount"] += item_amount
            line_items[group_key]["billing_period"] = billing_period

    result = {
        "line_items": [],
        "total_revenue": 0.0
        }

    for entry in line_items.values():
        entry["amount"] = round(entry["amount"], 2)
        result["line_items"].append(entry)
        result["total_revenue"] += entry["amount"]
        result["total_revenue"] = round(result["total_revenue"], 2)
    return result

item_list = [
    {"idx": 1, "item_code": "Executive Desk (4*2)", "sales_description": "Dedicated Executive Desk", "qty": 10, "rate": "1000", "amount": "10000", "start_date": "2023-11-01", "stop_date": "2024-10-17"},
    {"idx": 2, "item_code": "Executive Desk (4*2)", "qty": "10", "rate": "1080", "amount": "10800", "start_date": "2024-10-18", "stop_date": "2025-10-31"},
    {"idx": 3, "item_code": "Executive Desk (4*2)", "qty": 15, "rate": "1080", "amount": "16200", "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 4, "item_code": "Executive Desk (4*2)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 5, "item_code": "Manager Cabin", "qty": 5, "rate": 5000, "amount": 25000, "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 6, "item_code": "Manager Cabin", "qty": 7, "rate": "5000", "amount": "35000", "start_date": "2024-12-15", "stop_date": "2025-10-31"},
    {"idx": 7, "item_code": "Manager Cabin", "qty": 10, "rate": 4600, "amount": 46000, "start_date": "2023-11-01", "stop_date": "2024-10-17"},
    {"idx": 8, "item_code": "Parking (2S)", "qty": 10, "rate": 1000, "amount": 10000, "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 9, "item_code": "Parking (2S)", "qty": 10, "rate": 0, "amount": 0, "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 10, "item_code": "Executive Desk (4*2)", "qty": "8", "rate": "1100", "amount": "8800", "start_date": "2024-11-15", "stop_date": "2025-01-31"},
    {"idx": 11, "item_code": "Manager Cabin", "qty": "3", "rate": "5200", "amount": "15600", "start_date": "2024-10-10", "stop_date": "2024-11-10"},
    {"idx": 12, "item_code": "Conference Table", "qty": 1, "rate": "20000", "amount": "20000", "start_date": "2024-11-05", "stop_date": "2024-11-20"},
    {"idx": 13, "item_code": "Parking (2S)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-15", "stop_date": "2025-02-28"},
    {"idx": 14, "item_code": "Reception Desk", "qty": 2, "rate": "7000", "amount": "14000", "start_date": "2024-11-01", "stop_date": "2025-03-31"},
    {"idx": 15, "item_code": "Reception Desk", "qty": 1, "rate": "7000", "amount": "7000", "start_date": "2024-11-10", "stop_date": "2024-11-25"},
    {"idx": 16, "item_code": "Breakout Area", "qty": 3, "rate": "3000", "amount": "9000", "start_date": "2024-01-01", "stop_date": "2024-01-31"}
]

bill = generate_monthly_bill(item_list, "2024-11")
print(bill)